<a href="https://colab.research.google.com/github/aadityane93/SortingLine-ml/blob/main/data_to_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Python Libraries

In [1]:
from google.colab import files
import numpy as np
import pandas as pd

# Setting Folder-id

In [2]:
#Setting Folder-id to import Files from Google Drive

#100hz
# folder_id = '1Goeo7w4XQ_BRZVN8zRX3t11Ollg811VB' #10 files
# folder_id = '1BZJfMkmr5uyyL1iZGPQen-B3VTYrJjD0' #100 files
# folder_id = '1bxp8LJq3826UeOR44M9ZiDFX61Dp-xSf' #all 1000 files

#400hz
# folder_id = '1t67fHYWy4DenBOEtt8woJY-WTEtni_lZ' #all 1000 files

#800hz
folder_id = '1VjStpb6GkRMpyM5kRqFdk7i9ep9aSUni' #10 files

# Importing files

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

folder_list = drive.ListFile({'q': "'{}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false".format(folder_id)}).GetList()


for folder in folder_list:
    folder_title = folder['title']
    print(f"Processing folder: {folder_title}")
    file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder['id'])}).GetList()

    for index, file in enumerate(file_list):
        file_title = file['title']
        new_file_name = f"{folder_title}-{index}"
        print(f"Downloading {file_title} as {new_file_name}")
        file.GetContentFile(new_file_name)

Processing folder: riO
Processing folder: wiO
Processing folder: niO
Processing folder: biO


# Setting No. of Files and Train Percentage

In [4]:
########################################################## INDEX ################################################

index = 10   # __No. of files from each color


########################################################## Train Percentage ################################################


#Train Percentage
test_train_split_percentage = 50   # __% percent of data files will be used to train the model

# Info on test/train files

In [5]:
print ("total no. of files for each color  - ", index)
print ("total no. of files  - ", index*4)
print (" ")

#Index to seperarate the read data into train and test dataframes
index_train = (test_train_split_percentage/100)* index
index_train = int(index_train)
index_test_percentage = 100 - test_train_split_percentage
index_test = (index_test_percentage/100)* index
index_test = int(index_test)

print (f"Percent of training data - {test_train_split_percentage}%")
print (f"No. of files for training - {index_train}")
print (" ")
print (f"Percent of testing data - {index_test_percentage}%")
print (f"No. of files for testing - {index_test}")

total no. of files for each color  -  10
total no. of files  -  40
 
Percent of training data - 50%
No. of files for training - 5
 
Percent of testing data - 50%
No. of files for testing - 5


# Reading Files and Assigning Dataframe

In [6]:
def read_file(filename):
    return pd.read_csv((filename), sep=',')

# Dictonaries to read the files
white_train = {}
white_test = {}
blue_train = {}
blue_test = {}
red_train = {}
red_test = {}
nio_train = {}
nio_test = {}

#Assigning df
white_train_df = pd.DataFrame()
white_test_df = pd.DataFrame()
blue_train_df = pd.DataFrame()
blue_test_df = pd.DataFrame()
red_train_df = pd.DataFrame()
red_test_df = pd.DataFrame()
nio_train_df = pd.DataFrame()
nio_test_df = pd.DataFrame()

#Reading files for training
for j in range (index_train):
  white_train[j] = read_file(f"wiO-{j}")
  blue_train[j] = read_file(f"biO-{j}")
  red_train[j] = read_file(f"riO-{j}")
  nio_train[j] = read_file(f"niO-{j}")
  print(j)

print("\n")
#Reading files for testing
for i in range (index_test):
  white_test[i] = read_file(f"wiO-{i+index_train}")
  blue_test[i] = read_file(f"biO-{i+index_train}")
  red_test[i] = read_file(f"riO-{i+index_train}")
  nio_test[i] = read_file(f"niO-{i+index_train}")
  print (i+index_train)

0
1
2
3
4


5
6
7
8
9


In [7]:
white_train

{0:            Datum                         Uhrzeit  I_In   V_In  P_In(W)  \
 0     2024-05-03  2024-05-03 11:02:08.820138:820  0.25  11.65   2.9125   
 1     2024-05-03  2024-05-03 11:02:08.820466:820  0.29  11.70   3.3930   
 2     2024-05-03  2024-05-03 11:02:08.824308:824  0.29  11.70   3.3930   
 3     2024-05-03  2024-05-03 11:02:08.825066:825  0.29  11.70   3.3930   
 4     2024-05-03  2024-05-03 11:02:08.825513:825  0.29  11.70   3.3930   
 ...          ...                             ...   ...    ...      ...   
 4981  2024-05-03  2024-05-03 11:02:15.058224:058  0.46  11.65   5.3590   
 4982  2024-05-03  2024-05-03 11:02:15.058369:058  0.46  11.65   5.3590   
 4983  2024-05-03  2024-05-03 11:02:15.062070:062  0.50  11.48   5.7400   
 4984  2024-05-03  2024-05-03 11:02:15.062231:062  0.50  11.48   5.7400   
 4985  2024-05-03  2024-05-03 11:02:15.062375:062  0.50  11.48   5.7400   
 
       I_Out(A)  V_Out(V)  P_Out(W)  Temp(°C)  Energie_In(Wh)  Energie_Out(Wh)  
 0         0.2

# Target Dataframe

In [8]:
# Color df for Train Dataset
print("White Train ", len(white_train), " ", "Blue Train ", len(blue_train), " ", "Red Train ", len(red_train), " ", "Nio Train ", len(nio_train))

df_color_train = pd.DataFrame()
for i in range(len(white_train)):
  df_color_train = pd.concat([df_color_train, pd.Series(0)])
for i in range(len(blue_train)):
  df_color_train = pd.concat([df_color_train, pd.Series(1)])
for i in range(len(red_train)):
  df_color_train = pd.concat([df_color_train, pd.Series(2)])
for i in range(len(nio_train)):
  df_color_train = pd.concat([df_color_train, pd.Series(3)])
df_color_train = df_color_train.astype(int)
df_color_train.columns = ['color']


# Color df for Test Dataset
print("White Test ", len(white_test), " ", "Blue Test ", len(blue_test), " ", "Red Test ", len(red_test), " ", "Nio Test ", len(nio_test))
df_color_test = pd.DataFrame()
for i in range(len(white_test)):
  df_color_test = pd.concat([df_color_test, pd.Series(0)])
for i in range(len(blue_test)):
  df_color_test = pd.concat([df_color_test, pd.Series(1)])
for i in range(len(red_test)):
  df_color_test = pd.concat([df_color_test, pd.Series(2)])
for i in range(len(nio_test)):
  df_color_test = pd.concat([df_color_test, pd.Series(3)])
df_color_test = df_color_test.astype(int)
df_color_test.columns = ['color']

White Train  5   Blue Train  5   Red Train  5   Nio Train  5
White Test  5   Blue Test  5   Red Test  5   Nio Test  5


In [9]:
print(df_color_train.shape)
print(df_color_test.shape)

(20, 1)
(20, 1)


# Train/Test Dataframe for each colors

In [10]:
# Merging df of different Colors into a single df for Train Dataset

q= 0
r= index_train + index_test
s = index_train + index_test + r
t = index_train + index_test + s


for j in range(index_train):
  print (' q - ',q,' r - ',r,' s - ',s,' t - ',t)
  white_train[j] = pd.DataFrame(white_train[j])
  white_train[j]['fileno'] = q
  white_train_df = pd.concat([white_train_df, white_train[j]])

  blue_train[j] = pd.DataFrame(blue_train[j])
  blue_train[j]['fileno'] = r
  blue_train_df = pd.concat([blue_train_df, blue_train[j]])

  red_train[j] = pd.DataFrame(red_train[j])
  red_train[j]['fileno'] = s
  red_train_df = pd.concat([red_train_df, red_train[j]])

  nio_train[j] = pd.DataFrame(nio_train[j])
  nio_train[j]['fileno'] = t
  nio_train_df = pd.concat([nio_train_df, nio_train[j]])

  q = q + 1
  r = r + 1
  s = s + 1
  t = t + 1



# Merging df of different Colors into a single df for Test Dataset

q= index_train
r= r
s = s
t = t

print (" \n")

for j in range(index_test):
  print (' q - ',q,' r - ',r,' s - ',s,' t - ',t)
  white_test[j] = pd.DataFrame(white_test[j])
  white_test[j]['fileno'] = q
  white_test_df = pd.concat([white_test_df, white_test[j]])

  blue_test[j] = pd.DataFrame(blue_test[j])
  blue_test[j]['fileno'] = r
  blue_test_df = pd.concat([blue_test_df, blue_test[j]])

  red_test[j] = pd.DataFrame(red_test[j])
  red_test[j]['fileno'] = s
  red_test_df = pd.concat([red_test_df, red_test[j]])

  nio_test[j] = pd.DataFrame(nio_test[j])
  nio_test[j]['fileno'] = t
  nio_test_df = pd.concat([nio_test_df, nio_test[j]])

  q = q + 1
  r = r + 1
  s = s + 1
  t = t + 1

 q -  0  r -  10  s -  20  t -  30
 q -  1  r -  11  s -  21  t -  31
 q -  2  r -  12  s -  22  t -  32
 q -  3  r -  13  s -  23  t -  33
 q -  4  r -  14  s -  24  t -  34
 

 q -  5  r -  15  s -  25  t -  35
 q -  6  r -  16  s -  26  t -  36
 q -  7  r -  17  s -  27  t -  37
 q -  8  r -  18  s -  28  t -  38
 q -  9  r -  19  s -  29  t -  39


# Target attribute

In [11]:
#Assigning color to the the dataframe
white_train_df ['color'] = 0
blue_train_df ['color'] = 1
red_train_df ['color'] = 2
nio_train_df ['color'] = 3

white_test_df ['color'] = 0
blue_test_df ['color'] = 1
red_test_df ['color'] = 2
nio_test_df ['color'] = 3

# Final features DataFrames

In [12]:
df_train = pd.concat([white_train_df, blue_train_df, red_train_df, nio_train_df])
df_test = pd.concat([white_test_df, blue_test_df, red_test_df, nio_test_df])

# Resetting Index on Feature & Target DataFrames

In [13]:
df_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)
df_color_train.reset_index(drop=True, inplace=True)
df_color_test.reset_index(drop=True, inplace=True)

# Saving the final Feature and Target Dataframes

In [14]:
df_train_file = f'df_train-({index}-file-per-each-color).csv'
print(df_train_file)
df_train.to_csv(df_train_file, sep=',')
files.download(df_train_file)

df_test_file = f'df_test-({index}-file-per-each-color).csv'
print(df_test_file)
df_test.to_csv(df_test_file, sep=',')
files.download(df_test_file)

df_color_train_file = f'df_color_train-({index}-file-per-each-color).csv'
print(df_color_train_file)
df_color_train.to_csv(df_color_train_file, sep=',')
files.download(df_color_train_file)

df_color_test_file = f'df_color_test-({index}-file-per-each-color).csv'
print(df_color_test_file)
df_color_test.to_csv(df_color_test_file, sep=',')
files.download(df_color_test_file)

df_train-(10-file-per-each-color).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

df_test-(10-file-per-each-color).csv


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

df_color_test-(10-file-per-each-color).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>